### Implémentation de Exemplar-based inpainting

In [7]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

### Etape 1a. (Identify the fill front)

In [8]:
# Charger les images
image = Image.open("fleur.tif")
mask_border = Image.open("mask_border.png")
filled_region_mask = Image.open("filled_region_mask.png")
fleurs = Image.open("fleur.tif")

# Convertir l'image en tableau NumPy
mask_border = np.array(mask_border)/255
filled_region_mask = np.array(filled_region_mask)/255
image_np = np.mean(np.array(image),axis=2)/255
H,W = mask_border.shape
print(H,W)

FileNotFoundError: [Errno 2] No such file or directory: 'fleur.tif'

### Etape 1.b Compute priorities

In [10]:
def is_in_image(i,j,image):
    H,W = image.shape
    return (0<=i<H) and (0<=j<W)

def update_confidence(border_map,confidence_map,filled_region_mask,patch_size):
    delta = patch_size//2
    border_points = np.column_stack(np.where(border_map==1))
    for (y,x) in border_points:
        confidence = 0
        for dx in range(-delta,delta+1):
            for dy in range(-delta,delta+1):
                ny, nx = y+dy, x+dx
                if (is_in_image(ny,nx,border_map)) and filled_region_mask[ny,nx]==1:
                    confidence += confidence_map[ny,nx]
        confidence /= patch_size**2
        confidence_map[y,x] = confidence

def compute_gradient(x,y,image):
    H,W = image.shape
    x_minus_1 = max(0,x-1)
    x_plus_1 = min(x+1,W-1)
    y_plus_1 = min(H-1,y+1)
    y_minus_1 = max(0,y-1)
    
    grad_x = image[y,x_plus_1] - image[y,x_minus_1]
    grad_y = image[y_plus_1,x] - image[y_minus_1,x]
    return grad_x, grad_y

#TODO Implémenter une formule moins grossière pour la donnée
def compute_data(border_map,image):
    border_points = np.column_stack(np.where(border_map==1))
    border_points_with_gradients = []
    for (y,x) in border_points:
        grad_x, grad_y = compute_gradient(x,y,image)
        data_value = np.sqrt(grad_x**2+grad_y**2)
        border_points_with_gradients.append(((y,x),data_value))
    return border_points_with_gradients

### Etape 2.a Find the patch with maximum priority

In [17]:
def compute_next_pixel_by_priority(border_map,confidence_map,filled_region_mask,image,patch_size):
    update_confidence(border_map,confidence_map,filled_region_mask,patch_size)
    border_points_with_gradients = compute_data(border_map,image)
    xmax, ymax = 0, 0
    priority_max = -np.inf
    for ((y, x), data) in border_points_with_gradients:
        priority = confidence_map[y,x] * data
        if priority > priority_max:
            xmax, ymax = x, y
            priority_max = priority
    #print(priority_max, xmax, ymax, filled_region_mask[ymax,xmax])
    return (xmax,ymax)

def compute_mask_around_pixel(x,y,filled_region_mask,patch_size):
    mask = np.zeros((patch_size,patch_size))
    delta = patch_size//2
    for dx in range(-delta,delta+1):
        for dy in range(-delta,delta+1):
            ny, nx = y+dy, x+dx
            if (is_in_image(ny,nx,filled_region_mask) and filled_region_mask[ny,nx]==1) :
                mask[dy+delta,dx+delta] = 1
    #print("Mask", np.sum(mask))
    return mask

### Etape 2.b Find the exemplar patch that minimises the SSD

In [13]:
#image_copy = np.copy(image_np)

def compute_MSE(target_x,target_y,source_patch,mask,image):
    patch_size = source_patch.shape[0]
    H, W = image.shape
    delta = patch_size//2
    x_min = max(0,target_x-delta)
    x_max = min(W,target_x+delta+1)
    y_min = max(0,target_y-delta)
    y_max = min(H,target_y+delta+1)
    target_patch = np.zeros_like(source_patch)
    target_patch[0:y_max-y_min,0:x_max-x_min] = image[y_min:y_max,x_min:x_max]
    diff_all = (target_patch-source_patch)**2
    diff = diff_all * mask
    num_valid_pixel = np.sum(mask)
    MSE = np.sum(diff) / num_valid_pixel
    return MSE
    
def is_outside(i,j,filled_region_mask,patch_size):
    delta = patch_size//2
    H, W = filled_region_mask.shape
    if i - delta < 0 or i + delta >= W or j - delta < 0 or j + delta >= H:
        return False
    considered_patch = filled_region_mask[j-delta:j+delta+1,i-delta:i+delta+1]
    return np.sum(considered_patch)==patch_size**2
    
def find_best_matching_patch(x,y,mask,filled_region_mask,image,patch_size):
    H,W = image.shape
    delta = patch_size//2
    min_MSE = float('inf')
    optimal_x, optimal_y = 0, 0
    for i in range(delta,W-delta):
        for j in range(delta,H-delta):
            if is_outside(i,j,filled_region_mask,patch_size):
                source_patch = image[j-delta:j+delta+1,i-delta:i+delta+1]
                current_MSE = compute_MSE(x,y,source_patch,mask,image)
                if (current_MSE<min_MSE):
                    min_MSE = current_MSE
                    optimal_x = i
                    optimal_y = j
    #print(optimal_x,optimal_y,min_MSE)
    return optimal_x, optimal_y

### Etape 2.c et 3. Copy image data from Ψqˆ to Ψpˆ + update confidence.

In [16]:
from scipy.ndimage import convolve
def copy_image_data(source_x,source_y,target_x,target_y,mask,filled_region_mask,image,confidence_map):
    patch_size = mask.shape[0]
    delta = patch_size//2
    for i in range(-delta,delta+1):
        for j in range(-delta,delta+1):
            current_source_x, current_source_y = source_x+i, source_y+j
            current_target_x, current_target_y = target_x+i, target_y+j
            if is_in_image(current_target_y,current_target_x,filled_region_mask) and mask[j+delta,i+delta]==0:
                image[current_target_y,current_target_x] = image[current_source_y,current_source_x]
                filled_region_mask[current_target_y,current_target_x] = 1
                confidence_map[current_target_y,current_target_x] = confidence_map[target_y,target_x]

def update_border(filled_region_mask):
    neighbor_kernel = np.array([[1,1,1],
                          [1,0,1],
                          [1,1,1]])
    not_filled_mask = (1-filled_region_mask).astype(bool)
    has_filled_neighbor = (convolve(filled_region_mask,neighbor_kernel,mode='constant',cval=0)>=1)
    new_border = (not_filled_mask) & (has_filled_neighbor)
    return new_border


### Final implementation

In [19]:
from tqdm import tqdm
# Charger les images
image = Image.open("yezhov.png")
mask_border = Image.open("mask_border.png")
filled_region_mask = Image.open("filled_region_mask.png")

# Convertir l'image en tableau NumPy
mask_border = np.array(mask_border)/255
filled_region_mask = np.array(filled_region_mask)/255
image = np.mean(np.array(image),axis=2)/255
initial_region_size = np.sum(filled_region_mask == 0)

H,W = mask_border.shape
print(np.sum(filled_region_mask == 0))
confidence_map = np.copy(filled_region_mask)
patch_size = 21
step = 0
with tqdm(total=initial_region_size) as pbar:
    while np.sum(filled_region_mask == 0) > 0:
        current_x, current_y = compute_next_pixel_by_priority(mask_border,confidence_map,filled_region_mask,image,patch_size)
        mask_patch = compute_mask_around_pixel(current_x,current_y,filled_region_mask,patch_size)
        source_x, source_y = find_best_matching_patch(current_x, current_y,mask_patch,filled_region_mask,image,patch_size)
        copy_image_data(source_x,source_y,current_x,current_y,mask_patch,filled_region_mask,image,confidence_map)
        mask_border = update_border(filled_region_mask)
        cv2.imwrite("border.png", mask_border * 255)
        cv2.imwrite("filled.png", filled_region_mask * 255)
        remaining = np.sum(filled_region_mask == 0)
        pbar.update(initial_region_size - remaining)
        initial_region_size = remaining
        #print(np.sum(filled_region_mask == 0))
        step += 1
cv2.imwrite("image_yezhov_inpaint.png", image * 255)

30306


100%|████████████████████████████████████| 30306/30306 [03:35<00:00, 140.54it/s]


True